## Download LAUS data from BLS 
  - state and county level labor market stats 
  - the [link](https://download.bls.gov/pub/time.series/la/la.series) to the data to be stored as laus.txt
  - or to find the format of series id [here](https://www.bls.gov/help/hlpforma.htm#LA)
  - [information about the data](https://download.bls.gov/pub/time.series/la/la.txt)
  
  
  - 'Registered users may request up to 20 years per query'

In [1]:
with open("BLS_API_KEY.txt", "r") as file:
    API_KEY = file.read()

In [2]:
## get all data series id to download remotely 

import pandas as pd

laus_series =  pd.read_csv("../OtherData/bls/laus_series.txt",
                           sep ='\t',
                           low_memory=False
                          )

In [3]:
## select data 

area_type_code = 'A' ## state 
measure_code = 3 ## UE rate 
seasonal = 'S' ## seasonal adjusted 

new_column_names = {col: col.strip().replace(' ', '_') for col in laus_series.columns}
laus_series.rename(columns=new_column_names, inplace=True)

series_ids = list((laus_series[(laus_series['area_type_code']==area_type_code) &
               (laus_series['measure_code']==measure_code) & 
               (laus_series['seasonal']==seasonal)]['series_id']))

series_ids  = [series_id.strip().replace(' ', '_') for series_id in series_ids]


In [4]:
import requests
import json

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": series_ids,
                   "startyear":"2001", 
                   "endyear":"2023",
                  "registrationkey":API_KEY})

p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', 
                  data=data, 
                  headers=headers)

json_data = json.loads(p.text)

table_data =  []
for series in json_data['Results']['series']:
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            table_data.append([seriesId,
                       year,
                       period,
                       value,
                       footnotes[0:-1]])
        

In [5]:
print('Length of the first extract: '+ str(len(table_data)))

Length of the first extract: 12000


In [6]:
data2 = json.dumps({"seriesid": series_ids,
                   "startyear":"1981", 
                   "endyear":"2000",
                  "registrationkey":API_KEY})

p2 = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', 
                  data=data, 
                  headers=headers)

json_data2 = json.loads(p2.text)

table_data2 =  []
for series in json_data2['Results']['series']:
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            table_data2.append([seriesId,
                       year,
                       period,
                       value,
                       footnotes[0:-1]])
        

In [7]:
data3 = json.dumps({"seriesid": series_ids,
                   "startyear":"1976", 
                   "endyear":"1980",
                  "registrationkey":API_KEY})

p3 = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', 
                  data=data, 
                  headers=headers)

json_data3 = json.loads(p3.text)

table_data3 =  []
for series in json_data3['Results']['series']:
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            table_data3.append([seriesId,
                       year,
                       period,
                       value,
                       footnotes[0:-1]])
        

In [8]:
print('Length of the second extract: '+ str(len(table_data3)))

Length of the second extract: 12000


In [10]:
## combine data obtained from various querries 

table_data = table_data+table_data2+table_data3

In [11]:
print('Total length of data: '+str(len(table_data)))

Total length of data: 36000


In [12]:
# Create a Pandas DataFrame from the extracted data
names = ["series_id","year","period","value","footnotes"]
df_all = pd.DataFrame(table_data, columns=names)

In [13]:
# Extract data from PrettyTable

df_all['state'] = df_all['series_id'].str[5:7].astype(int)

In [14]:
## how many states have data?

len(df_all['state'].unique())

50

In [15]:
df_all

,series_id,year,period,value,footnotes,state
0,LASST010000000000003,2020,M12,4.5,"Data were subject to revision on March 1, 2023.",1
1,LASST010000000000003,2020,M11,4.9,"Data were subject to revision on March 1, 2023.",1
2,LASST010000000000003,2020,M10,5.3,"Data were subject to revision on March 1, 2023.",1
3,LASST010000000000003,2020,M09,5.9,"Data were subject to revision on March 1, 2023.",1
4,LASST010000000000003,2020,M08,6.3,"Data were subject to revision on March 1, 2023.",1
...,...,...,...,...,...,...
35995,LASST550000000000003,2001,M05,4.2,,55
35996,LASST550000000000003,2001,M04,4.1,,55
35997,LASST550000000000003,2001,M03,4.0,,55
35998,LASST550000000000003,2001,M02,3.9,,55


In [16]:
df_all['date'] = pd.to_datetime(df_all['year'].astype(str) + df_all['period'], format='%YM%m')

In [17]:
df_all = df_all.drop(columns=['series_id','year','period','footnotes'])

In [18]:
df_all = df_all.rename(columns={'state':'statecode'})


,value,footnotes,state,date
0,4.5,"Data were subject to revision on March 1, 2023.",1,2020-12-01
1,4.9,"Data were subject to revision on March 1, 2023.",1,2020-11-01
2,5.3,"Data were subject to revision on March 1, 2023.",1,2020-10-01
3,5.9,"Data were subject to revision on March 1, 2023.",1,2020-09-01
4,6.3,"Data were subject to revision on March 1, 2023.",1,2020-08-01
...,...,...,...,...
35995,4.2,,55,2001-05-01
35996,4.1,,55,2001-04-01
35997,4.0,,55,2001-03-01
35998,3.9,,55,2001-02-01


In [19]:
df_all = df_all.drop_duplicates()
df_all

,value,footnotes,state,date
0,4.5,"Data were subject to revision on March 1, 2023.",1,2020-12-01
1,4.9,"Data were subject to revision on March 1, 2023.",1,2020-11-01
2,5.3,"Data were subject to revision on March 1, 2023.",1,2020-10-01
3,5.9,"Data were subject to revision on March 1, 2023.",1,2020-09-01
4,6.3,"Data were subject to revision on March 1, 2023.",1,2020-08-01
...,...,...,...,...
11995,4.2,,55,2001-05-01
11996,4.1,,55,2001-04-01
11997,4.0,,55,2001-03-01
11998,3.9,,55,2001-02-01


<DatetimeArray>
['2020-12-01 00:00:00', '2020-11-01 00:00:00', '2020-10-01 00:00:00',
 '2020-09-01 00:00:00', '2020-08-01 00:00:00', '2020-07-01 00:00:00',
 '2020-06-01 00:00:00', '2020-05-01 00:00:00', '2020-04-01 00:00:00',
 '2020-03-01 00:00:00',
 ...
 '2001-10-01 00:00:00', '2001-09-01 00:00:00', '2001-08-01 00:00:00',
 '2001-07-01 00:00:00', '2001-06-01 00:00:00', '2001-05-01 00:00:00',
 '2001-04-01 00:00:00', '2001-03-01 00:00:00', '2001-02-01 00:00:00',
 '2001-01-01 00:00:00']
Length: 240, dtype: datetime64[ns]